In [34]:
from graph4nlp.pytorch.modules.graph_construction import base

In [32]:
import copy
import torch
import random
import argparse
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

from graph4nlp.pytorch.data.data import to_batch
from graph4nlp.pytorch.datasets.mawps import MawpsDatasetForTree
from graph4nlp.pytorch.modules.graph_construction import DependencyBasedGraphConstruction
from graph4nlp.pytorch.modules.graph_embedding import *
from graph4nlp.pytorch.models.graph2tree import Graph2Tree
from graph4nlp.pytorch.modules.utils.tree_utils import Tree #prepare_oov

# from utils import convert_to_string, compute_tree_accuracy

In [33]:
import os
import random
import time
import pickle
import argparse
import copy

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
from torch.utils.data import DataLoader

from graph4nlp.pytorch.data.data import to_batch
from graph4nlp.pytorch.datasets.mawps import MawpsDatasetForTree
from graph4nlp.pytorch.modules.graph_construction import *
from graph4nlp.pytorch.modules.graph_embedding import *
from graph4nlp.pytorch.models.graph2tree import Graph2Tree
from graph4nlp.pytorch.modules.utils.tree_utils import Tree, VocabForAll
from examples.pytorch.math_word_problem.mawps.src.evaluation import convert_to_string, compute_tree_accuracy
import warnings
warnings.filterwarnings('ignore')




## concatenating all files for svamp


In [8]:
# import necessary libraries
import pandas as pd
import os
import glob
final_df = pd.DataFrame()
# use glob to get all the csv files
# in the folder
path = os.getcwd() 
path = path + "/examples/pytorch/math_word_problem/svamp/svamp_data/raw"
csv_files = glob.glob(os.path.join(path, "*.csv"))


# loop over the list of csv files
for f in csv_files:

    # read the csv file
    df = pd.read_csv(f)
    final_df = pd.concat([final_df,df])
    
    # print the location and filename
    #print('Location:', f)
    print('File Name:', f.split("\\")[-1])


print(len(final_df))



File Name: /Users/vaishnavipariti/Documents/sem5_project/graph4nlp/examples/pytorch/math_word_problem/svamp/svamp_data/raw/train0.csv
File Name: /Users/vaishnavipariti/Documents/sem5_project/graph4nlp/examples/pytorch/math_word_problem/svamp/svamp_data/raw/train1.csv
File Name: /Users/vaishnavipariti/Documents/sem5_project/graph4nlp/examples/pytorch/math_word_problem/svamp/svamp_data/raw/train3.csv
File Name: /Users/vaishnavipariti/Documents/sem5_project/graph4nlp/examples/pytorch/math_word_problem/svamp/svamp_data/raw/train2.csv
File Name: /Users/vaishnavipariti/Documents/sem5_project/graph4nlp/examples/pytorch/math_word_problem/svamp/svamp_data/raw/dev4.csv
File Name: /Users/vaishnavipariti/Documents/sem5_project/graph4nlp/examples/pytorch/math_word_problem/svamp/svamp_data/raw/dev0.csv
File Name: /Users/vaishnavipariti/Documents/sem5_project/graph4nlp/examples/pytorch/math_word_problem/svamp/svamp_data/raw/dev1.csv
File Name: /Users/vaishnavipariti/Documents/sem5_project/graph4nlp/e

In [11]:
req_df = final_df[['Question','Equation']]
display(req_df)

,Question,Equation
0,bobby ate some pieces of candy . then he ate n...,- number1 number0
1,bobby ate number0 pieces of candy . then he at...,+ number0 number1
2,bobby ate number0 pieces of candy . then he at...,+ + number0 number1 number2
3,bobby ate number0 pieces of candy . then he at...,- + number0 number1 number2
4,bobby had number0 pieces of candy . he ate num...,+ number1 number2
...,...,...
3949,there are number0 bananas in janice 's banana ...,/ number0 number2
3950,janice has number0 bottle caps that must be pu...,/ number0 number2
3951,chris is inviting number0 friends to a party ....,/ number1 number0
3952,keith has number0 marbles and number1 pencils ...,/ number0 number2


In [22]:
train_size = 0.8
valid_size=0.1

train_index = int(len(req_df)*train_size)

# shuffle data
req_df.sample(frac=1).reset_index()

df_train = req_df[0:train_index]
df_rem = req_df[train_index:]

valid_index = int(len(req_df)*valid_size)

df_valid = req_df[train_index:train_index+valid_index]
df_test = req_df[train_index+valid_index:]

In [23]:
train_p = path + '/train.csv'
test_p = path + '/test.csv'
valid_p = path + '/valid.csv'

In [26]:
df_test

,Question,Equation
1885,The cost of number0 feet of chain is number1 d...,* / number2 number1 number0
1886,Nancy has saved number0 cents from selling lem...,/ number0 100.0
1887,Paul made number0 dollars mowing lawns and num...,/ + number0 number0 number0
1888,Wendy 's old washing machine could only wash n...,/ + number1 number2 number0
1889,In fourth grade there were number0 students at...,- + number0 number2 number1
...,...,...
3949,there are number0 bananas in janice 's banana ...,/ number0 number2
3950,janice has number0 bottle caps that must be pu...,/ number0 number2
3951,chris is inviting number0 friends to a party ....,/ number1 number0
3952,keith has number0 marbles and number1 pencils ...,/ number0 number2


In [27]:
df_test.to_csv(test_p, header=None, index=None)
df_train.to_csv(train_p, header=None, index=None)
df_valid.to_csv(valid_p, header=None, index=None)

In [28]:
from graph4nlp.pytorch.modules.config import get_basic_args
from graph4nlp.pytorch.modules.utils.config_utils import update_values, get_yaml_config

def get_args():
    config = {'dataset_yaml': "./examples/pytorch/math_word_problem/svamp/config/svamp_dynamic_graph_construction.yml",# "./examples/pytorch/math_word_problem/mawps/config/new_dynamic_graphsage_undirected.yaml",
              'learning_rate': 1e-3,
              'gpuid': -1,
              'seed': 123,
              'init_weight': 0.08,
              'graph_type': 'dynamic',
              'weight_decay': 0,
              'max_epochs': 20,
              'min_freq': 1,
              'grad_clip': 5,
              'batch_size': 20,
              'share_vocab': True,
              'pretrained_word_emb_name': None,
              'pretrained_word_emb_url': None,
              'pretrained_word_emb_cache_dir': ".vector_cache",
              'checkpoint_save_path': "./checkpoint_save",
              'beam_size': 4
              }
    our_args = get_yaml_config(config['dataset_yaml'])
    template = get_basic_args(graph_construction_name=our_args["graph_construction_name"],
                              graph_embedding_name=our_args["graph_embedding_name"],
                              decoder_name=our_args["decoder_name"])
    update_values(to_args=template, from_args_list=[our_args, config])
    return template

# show our config
cfg_g2t = get_args()
from pprint import pprint
pprint(cfg_g2t)

{'batch_size': 20,
 'beam_size': 4,
 'checkpoint_save_path': './checkpoint_save',
 'dataset_yaml': './examples/pytorch/math_word_problem/svamp/config/svamp_dynamic_graph_construction.yml',
 'decoder_args': {'rnn_decoder_private': {'max_decoder_step': 35,
                                          'max_tree_depth': 8,
                                          'use_sibling': False},
                  'rnn_decoder_share': {'attention_type': 'uniform',
                                        'dropout': 0.3,
                                        'fuse_strategy': 'concatenate',
                                        'graph_pooling_strategy': None,
                                        'hidden_size': 300,
                                        'input_size': 300,
                                        'rnn_emb_input_size': 300,
                                        'rnn_type': 'lstm',
                                        'teacher_forcing_rate': 1.0,
                                 

In [36]:
class Mawps:
    def __init__(self, opt=None):
        super(Mawps, self).__init__()
        self.opt = opt

        seed = self.opt["seed"]
        print(seed)
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        print(opt["gpuid"])
        if self.opt["gpuid"] == -1:
            self.device = torch.device("cpu")
        else:
            self.device = torch.device("cuda:{}".format(self.opt["gpuid"]))

        self.use_copy = self.opt["decoder_args"]["rnn_decoder_share"]["use_copy"]
        self.use_share_vocab = self.opt["graph_construction_args"]["graph_construction_share"]["share_vocab"]
        self.data_dir = self.opt["graph_construction_args"]["graph_construction_share"]["root_dir"]

        self._build_dataloader()

    def _build_dataloader(self):
        graph_type = self.opt["graph_construction_args"]["graph_construction_share"]["graph_type"]
        enc_emb_size = self.opt["graph_construction_args"]["node_embedding"]["input_size"]
        tgt_emb_size = self.opt["decoder_args"]["rnn_decoder_share"]["input_size"]
        topology_subdir = self.opt["graph_construction_args"]["graph_construction_share"]["topology_subdir"]
        if graph_type == "dependency":
            dataset = MawpsDatasetForTree(root_dir=self.data_dir,
                                         topology_builder=DependencyBasedGraphConstruction,
                                         topology_subdir=topology_subdir, 
                                         edge_strategy=self.opt["graph_construction_args"]["graph_construction_private"]["edge_strategy"],
                                         graph_type='static',
                                         share_vocab=self.use_share_vocab, 
                                         enc_emb_size=enc_emb_size,
                                         dec_emb_size=tgt_emb_size,
                                         min_word_vocab_freq=self.opt["min_freq"],
                                         pretrained_word_emb_name=self.opt["pretrained_word_emb_name"],
                                         pretrained_word_emb_url=self.opt["pretrained_word_emb_url"], 
                                         pretrained_word_emb_cache_dir=self.opt["pretrained_word_emb_cache_dir"])

        elif graph_type == "constituency":
            dataset = MawpsDatasetForTree(root_dir=self.data_dir,
                                         topology_builder=ConstituencyBasedGraphConstruction,
                                         topology_subdir=topology_subdir, 
                                         edge_strategy=self.opt["graph_construction_args"]["graph_construction_private"]["edge_strategy"],
                                         graph_type='static',
                                         share_vocab=self.use_share_vocab, 
                                         enc_emb_size=enc_emb_size,
                                         dec_emb_size=tgt_emb_size,
                                         min_word_vocab_freq=self.opt["min_freq"],
                                         pretrained_word_emb_name=self.opt["pretrained_word_emb_name"],
                                         pretrained_word_emb_url=self.opt["pretrained_word_emb_url"], 
                                         pretrained_word_emb_cache_dir=self.opt["pretrained_word_emb_cache_dir"])
                                         
        elif graph_type == "node_emb":
            dataset = MawpsDatasetForTree(root_dir=self.data_dir, 
                                         word_emb_size=enc_emb_size,
                                         topology_builder=NodeEmbeddingBasedGraphConstruction,
                                         topology_subdir=topology_subdir, 
                                         graph_type='dynamic',
                                         dynamic_graph_type=graph_type, 
                                         edge_strategy=self.opt["graph_construction_args"]["graph_construction_private"]["edge_strategy"],
                                         share_vocab=self.use_share_vocab, 
                                         enc_emb_size=enc_emb_size,
                                         dec_emb_size=tgt_emb_size,
                                         min_word_vocab_freq=self.opt["min_freq"],
                                         pretrained_word_emb_name=self.opt["pretrained_word_emb_name"],
                                         pretrained_word_emb_url=self.opt["pretrained_word_emb_url"], 
                                         pretrained_word_emb_cache_dir=self.opt["pretrained_word_emb_cache_dir"])
    
        elif graph_type == "node_emb_refined":
            dynamic_init_graph_type = self.opt["graph_construction_args"]["graph_construction_private"]["dynamic_init_graph_type"]
            if dynamic_init_graph_type is None or dynamic_init_graph_type == 'line':
                dynamic_init_topology_builder = None
            elif dynamic_init_graph_type == 'dependency':
                dynamic_init_topology_builder = DependencyBasedGraphConstruction
            elif dynamic_init_graph_type == 'constituency':
                dynamic_init_topology_builder = ConstituencyBasedGraphConstruction
            else:
                # dynamic_init_topology_builder
                raise RuntimeError('Define your own dynamic_init_topology_builder')
            dataset = MawpsDatasetForTree(root_dir=self.data_dir,
                                         word_emb_size=enc_emb_size,
                                         topology_builder=NodeEmbeddingBasedRefinedGraphConstruction,
                                         topology_subdir=topology_subdir,
                                         graph_type='dynamic',
                                         dynamic_graph_type=graph_type,
                                         share_vocab=self.use_share_vocab,
                                         enc_emb_size=enc_emb_size, 
                                         dec_emb_size=tgt_emb_size,
                                         dynamic_init_topology_builder=dynamic_init_topology_builder,
                                         min_word_vocab_freq=self.opt["min_freq"],
                                         pretrained_word_emb_name=self.opt["pretrained_word_emb_name"],
                                         pretrained_word_emb_url=self.opt["pretrained_word_emb_url"], 
                                         pretrained_word_emb_cache_dir=self.opt["pretrained_word_emb_cache_dir"])                                         
        else:
            raise NotImplementedError

        self.train_data_loader = DataLoader(dataset.train, batch_size=self.opt["batch_size"], shuffle=True, num_workers=1,
                                           collate_fn=dataset.collate_fn)
        self.test_data_loader = DataLoader(dataset.test, batch_size=1, shuffle=False, num_workers=1,
                                          collate_fn=dataset.collate_fn)
        self.valid_data_loader = DataLoader(dataset.val, batch_size=1, shuffle=False, num_workers=1,
                                          collate_fn=dataset.collate_fn)
        self.src_vocab = dataset.src_vocab_model
        self.tgt_vocab = dataset.tgt_vocab_model
        if self.use_share_vocab:
            self.share_vocab = dataset.share_vocab_model
        self.vocab_model = VocabForAll(in_word_vocab=self.src_vocab, out_word_vocab=self.tgt_vocab, share_vocab=self.share_vocab)




In [37]:

if __name__ == "__main__":
#     from examples.pytorch.math_word_problem.mawps.src.config import get_args
    start = time.time()
    runner = Mawps(opt=get_args())
#     best_acc = runner.train()

    end = time.time()
    print("total time: {} minutes\n".format((end - start)/60))

123
-1
total time: 0.17891523440678914 minutes



In [38]:
# Load the file
data = torch.load("./examples/pytorch/math_word_problem/svamp/svamp_data/processed/NodeEmbGraph/data.pt")

In [56]:
data

{'train': [<graph4nlp.pytorch.data.dataset.Text2TreeDataItem at 0x2a84a1310>,
  ...],
 'test': [<graph4nlp.pytorch.data.dataset.Text2TreeDataItem at 0x29f9a3bb0>,
  ...],
 'val': [<graph4nlp.pytorch.data.dataset.Text2TreeDataItem at 0x2a24cd850>,
  ...]}

In [51]:
list(data.keys())[2]

'val'

In [57]:
build_topology(data['train'][1])

AttributeError: 'Text2TreeDataItem' object has no attribute 'build_topology'

In [58]:
from graph4nlp.pytorch.data.dataset import  TextToTreeDataset as tt

In [66]:
tt.collate_fn(data['train'])

{'graph_data': <graph4nlp.pytorch.data.data.GraphData at 0x2a19116d0>,
 'dec_tree_batch': [<graph4nlp.pytorch.modules.utils.tree_utils.Tree at 0x2a1911370>,
  ...],
 'original_dec_tree_batch': ['- number1 number0\n',
  '+ number0 number1\n',
  '+ + number0 number1 number2\n',
  '- + number0 number1 number2\n',
  '+ number1 number2\n',
  '- number0 + number1 number2\n',
  '- - number0 number1 number2\n',
  '- number0 number1\n',
  '- number0 number1\n',
  '- number2 + number0 number1\n',
  '- number1 number0\n',
  '- number0 number1\n',
  '- number0 number1\n',
  '+ + number0 number1 number2\n',
  '+ number0 number2\n',
  '/ number0 number1\n',
  '+ + number0 number1 number2\n',
  '+ number0 number1\n',
  '- number0 number1\n',
  '+ - number2 number1 number0\n',
  '- number0 number1\n',
  '- number1 number0\n',
  '- number0 number1\n',
  '- number0 number1\n',
  '+ number0 number1\n',
  '- + number0 number0 number1\n',
  '+ + number0 number0 number1\n',
  '+ + number0 number1 number2\n'

In [69]:
tt.vectorization(tt,data_items = data['train'])

AttributeError: type object 'TextToTreeDataset' has no attribute 'src_vocab_model'